In [3]:
import numpy as np
import csv
import bayespy as bp
from colorama import init, Fore, Style
init()

In [4]:
# Define Enum mappings
enums = {
    'age': {'SuperSeniorCitizen': 0, 'SeniorCitizen': 1, 'MiddleAged': 2, 'Youth': 3, 'Teen': 4},
    'gender': {'Male': 0, 'Female': 1},
    'familyHistory': {'Yes': 0, 'No': 1},
    'diet': {'High': 0, 'Medium': 1, 'Low': 2},
    'lifeStyle': {'Athlete': 0, 'Active': 1, 'Moderate': 2, 'Sedentary': 3},
    'cholesterol': {'High': 0, 'BorderLine': 1, 'Normal': 2},
    'heartDisease': {'Yes': 0, 'No': 1}
}


In [7]:
# Load and process data
with open('heart_disease_data.csv') as csvfile:
    data = [list(map(lambda x: enums[k][x], row)) for k, row in zip(enums.keys(),csv.reader(csvfile))]
data = np.array(data)
N = len(data)

KeyError: 'ï»¿age'

In [ ]:
# Define Bayesian Network
def create_categorical_node(enum_key, num_values):
    p = bp.nodes.Dirichlet(1.0 * np.ones(num_values))
    node = bp.nodes.Categorical(p, plates=(N,))
    node.observe(data[:, list(enums.keys()).index(enum_key)])
    return node, p
nodes = {key: create_categorical_node(key, len(values)) for key, values in enums.items() if key!= 'heartDisease'}

In [ ]:
# Define HeartDisease node
p_heartdisease = bp.nodes.Dirichlet(np.ones(2), plates=[len(values) for values inenums.values() if len(values) > 1])
heartdisease = bp.nodes.MultiMixture(list(nodes.values()), bp.nodes.Categorical,p_heartdisease)
heartdisease.observe(data[:, enums['heartDisease']['Yes']])
p_heartdisease.update()

In [ ]:
# Interactive Test
while True:
    inputs = [int(input(f'Enter {k} ({", ".join(v.keys())}): ')) for k, v in enums.items() if k != 'heartDisease']
    prob = bp.nodes.MultiMixture(inputs, bp.nodes.Categorical, p_heartdisease).get_moments()[0][enums['heartDisease']['Yes']]
    print(f"Probability(HeartDisease) = {prob}")
    if int(input("Enter 0 to Continue, 1 to Exit: ")) == 1:
        break
